# Calcular con conjuntos

¿Cómo podemos calcular el **rango** de una función sobre un **conjunto** $X$? Es decir,


$$\qquad \mathrm{rango}(f; X) := \{f(x): x \in X\}.$$

La **idea fundamental** es que definiremos operaciones sobre conjuntos, tal que el resultado contenga *todos los valores posibles*, operando con cualesquiera miembros del conjunto o de los conjuntos.

### Intervalos

Los subconjuntos más sencillos de $\mathbb{R}$ son los **intervalos** (siempre los tomaremos cerrados):

$$X = [a, b] := \{x : a \le x \le b \} \subseteq \mathbb{R}. $$

Vamos a definir funciones sobre intervalos de tal forma que *extiendan* la definición usual para los números reales.

Por ejemplo, ¿cómo podemos definir $X^2$? Lo definiremos como $X^2 := \{x^2: x \in X \}$.

Para $X = [1, 2]$, es evidente que $X^2 = [1, 4]$.

Pero para $X = [-1, 2]$ es menos evidente.
La solución completa es

\begin{align}
[a, b]^2 &:= [a^2, b^2] & \text{if } a \ge 0 \\ 
&:= [0, \max(a^2, b^2)]  &  \text{if } a < 0 \text{ and } b > 0\\
&:= [b^2, a^2] & \text{if } a < b < 0
\end{align}

En general, tratamos por separado cada pedazo monótono.

## Operaciones binarias

Para dos intervalos $X$ y $Y$, definimos

$X + Y := \{x + y: x \in X, y \in Y \}$,

etc.

Es evidente que  $$[a, b] + [c, d] = [a + c, b + d].$$

De la misma forma,

$$[a, b] - [c, d] = [a - d, b - c].$$
(Nota que el orden de los términos es diferente aquí.)

Esto lo repetimos para cada función. Entonces tenemos el siguiente

**Teorema**

- Si $f$ se define por una composición de operaciones unarias y binarias [es decir, $f$ es "factorable"] y $X$ es un interval, entonces 

    >  $f(X)$ contiene el rango verdadero de $f$
    
Aquí, $f(X)$ quiere decir "aplicar las extensiones intervalares de las funciones en $f$".

- Nota que la aritmética de intervalos da a menudo un resultado que es una **sobre-estimación** del rango verdadero, e.g.

$$[0, 1] - [0, 1] = [-1, 1].$$

Esto es el **efecto de dependencia**.

#### Ejercicio

Calcula $f(X)$ para $f(x) = x^2 - 2x$ y $X = [-2, 1]$.

## Implementacion de intervalos in Julia

Julia permite definir tipos compósitos que contienen datos de diferentes tipos, y definir operaciones sobre ellos.

In [ ]:
struct SimpleInterval
    lo::Float64
    hi::Float64
end

SimpleInterval(a) = SimpleInterval(a, a)

In [ ]:
a = SimpleInterval(0.1)
b = SimpleInterval(0.2)  

In [ ]:
import Base: +, show

show(io::IO, x::SimpleInterval) = 
    print(io, "[$(x.lo), $(x.hi)]")

+(x::SimpleInterval, y::SimpleInterval) = 
    SimpleInterval(x.lo + y.lo, x.hi + y.hi) 

In [ ]:
a = SimpleInterval(0.1, 0.1) 
b = SimpleInterval(0.2, 0.2)
a + b

In [ ]:
@code_native a + b

### Redondeo dirigido

Para garantizar que los resultados sean correctos con aritmética de punto flotante, se emple **redondeo dirigido** para garantizar que el resultado esté correctamente contenido. 

### Funciones 

Las funciones monótonas son fáciles *en principio*:


$$\exp([a, b]) := [\downarrow (\exp a), \uparrow(\exp b)]$$


Pero lograr un redondeo correcto está difícil: biblioteca (`CRlibm.jl`)


# IntervalArithmetic.jl

Junto con Luis Benet (Instituto de Ciencias Físicas, UNAM), hemos desarrollado un paquete para aritmética de intervalos en Julia:

In [ ]:
# Pkg.add("IntervalArithmetic")

using IntervalArithmetic

In [ ]:
@format full

X = interval(0.1)  # crea un intervalo "thin" (delgado -- de diámetro 0)

In [ ]:
diam(X)  # diámetro del intervalo

In [ ]:
big(X)

In [ ]:
0.1..0.1  # otra manera de crear intervalos: garantiza que se incluyan los dos puntos finales,
# tratados como números reales

In [ ]:
(0.1..0.2) + (0.3..0.4)

In [ ]:
@interval sin(0.1) + cos(0.2)^2   # evaluar expresión de forma garantizada

In [ ]:
@format standard 
IntervalBox(1..2, 3..4)   # caja en 2 dimensiones, producto Cartesiano de dos intervalos

(1..2) × (3..4)

## Aplicación: encerrar funciones de forma rigurosa

El evaluar la extensión intervalar de una función sobre un intervalo da un encierro, pero que puede ser una sobre-estimación.

Podemos mejorar esto al partir el intervalo en pedazos, lo cual se llama "desmenuzar" el interval ("mince"):

In [ ]:
function mince(x::Interval, n) 
    nodes = linspace(x.lo, x.hi, n+1)
    
    return [Interval(nodes[i], nodes[i+1]) for i in 1:length(nodes) - 1]
end

In [ ]:
f(x) = cos(x) + 0.5*sin(2*x)

function image(f, X)
    II = f(X)
   
    if II.lo == -Inf
        II = -100..(II.hi)
    end
    
    return II
end

In [ ]:
intervals = mince(-5..5, 10)
    
images = image.(f, intervals)

In [ ]:
using Interact, Plots, IntervalArithmetic; gr()

In [ ]:
function bound_function(f, X, N ,ylims=(-2,2))  
    @manipulate for N in slider(1:200, value=1)
        intervals = mince(X, N)    
        images = image.(f, intervals)

        boxes = [IntervalBox(intervals[i], images[i]) for i in 1:length(intervals)]
        contains_zero = [0 ∈ images[i] for i in 1:length(images)]

        if length(boxes[contains_zero]) > 0
            plot(boxes[contains_zero], ylim=ylims)   # uses "plot recipe" for IntervalBoxes
        end

        if length(boxes[.!(contains_zero)]) > 0
            plot!(boxes[.!(contains_zero)], ylim=ylims)  
        end
        plot!(f)
    end
end

In [ ]:
f(x) = cos(x) + 0.5*sin(2*x)

bound_function(f, -5..5,  50, (-2,2))

In [ ]:

f(x) = (1/50)*log(abs(3*(1-x) + 1)) + x^2 + 1

bound_function(f, 0.8..2.0, 100, (1,4))


In [ ]:
bound_function(x->sin(1/x), 0.01..0.2, (-1,1))

## Aplicación: Encontrar raíces al excluir regiones

Podemos utilizar intervalos para encontrar raíces. La idea es que si podemos encontrar un intervalo $X$ tal que $0 \notin f(X)$, entonces forzosamente no puede haber ningún $x \in X$ con $f(x) = 0$, y por lo tanto no hay ninguna raíz de $f$ en $X$.

In [ ]:
using IntervalArithmetic

In [ ]:
f(x) = x^2 - 2 

In [ ]:
@format standard
X = 3..4 

In [ ]:
typeof(X)   

In [ ]:
X = 3..4

X^2

In [ ]:
f(y) = y^2 - 2   # standard Julia function

f(X)   # automatically gives natural interval extension

In [ ]:
f(3..∞)  

Esto ¡nos da una **demostración rigurosa** que $f$ no tiene ninguna raíz en $[3, \infty]$! Esto es simplemente imposible con las técnicas usuales.

## Varias raíces: bisección

Si una función tiene múltiples raíces, quisiéramos poder encontrar *todas ellas*.  Esto está imposible con los métodos usuales.

Una manera de proceder es el desmenuzar el intervalo y eliminar subintervalos en los cuales *no* hay raíz. Para hacerlo, desarrollamos un árbol de intervalos con el siguiente algoritmo:

[0] Empezar con una lista de trabajo $L := [X]$; y una lista de raíces $R := [ ]$.


[1] Agarrar el último elemento de $L$ como $X$.


[2] Si $\mathrm{diam}(X) < \epsilon$, empujar $X$ a $R$.


[3] Si $0 \notin f(X)$, borrar $X$.


[4] Bisectar $X$ y empujar los resultantes $X_1$ and $X_2$ a $L$.

En Julia queda casi igual:

In [ ]:
function bisection(f, X, ϵ=1e-3)
    L = [X]; R = [ ]

    while !isempty(L)
        X = pop!(L)

        if diam(X) < ϵ 
            push!(R, X)
            continue 
        endk

        if 0 ∈ f(X) 
            X1, X2 = bisect(X)
            push!(L, X1, X2)
        end
    end 

    return R
end

In [ ]:
bisection(x->x^2 - 2, -∞..∞)

¡Nota que esta función sirve también para funciones en dimensión superior! Nunca se mencionó que $X$ fuera un intervalo. Casi todo funciona si es una caja, siempre y cuando existan las funciones `diam` y `bisect` para las cajas, y cambiemos `0 ∈ f(X)` por una función `contiene_cero`.

## Método de Newton intervalar

Hasta ahora, hemos *excluido* raíces. Hay una versión del método de Newton para intervalos, el cual permite garantizar que *sí* existe una raíz, y que es única.

La idea es tomar no sólo la recta tangente, si no **todos las pendientes $f'(X)$**. 
Esto echa mano del **teorema del valor medio** para funciones continuamente diferenciables, que dice que si hay una raíz $x^*$ en $X$ y $x$ es algún punto en X$, entonces

$$f(x) - f(x^*) = f(x) = (x - x^*) \cdot f'(\xi)$$

para algún $\xi \in X$.

Por lo tanto, tomando $x = m(X)$, vemos que el intervalo $m(X) - f(m(X)) \, / \, f'(X)$ debe contener la raíz.


Aquí, $f'(X)$ denota la extensión intervalar de la derivada de $f$, evaluada en $X$.

Afortunadamente, esto se puede calcular al utilizar diferenciación automática con intervalos.

Definimos

$\displaystyle \mathcal{N}_f(X) := m(X) - \frac{f([m(X)])}{f'(X)}$,

donde $m(X) := $ el punto medio de $X$

Hay un teorema que dice:

- $\mathcal{N}_f(X) \subset X \Longrightarrow \text{existe una única raíz en } X$

- $\mathcal{N}_f(X) \cap X = \emptyset \Longrightarrow \text{no hay raíz en } X$

Hay una extensión en la cual se permite que $f'(X)$ contenga $0$.